In [101]:
from pyspark import SQLContext,Row
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from nltk.corpus import stopwords
import re

file=sc.textFile("/Users/fusima/GitHub/sentiment-analysis-dictionary/data/training.1600000.processed.noemoticon.csv")\
        .map(lambda line: line.split('"')[1::2])\
        .sample(False,0.2,42)
    
# Alternativa per estrarre le sottostringhe contenute in una coppia di apici
#.map(lambda line: re.findall('"([^"]*)"',line))    

tweets = file.map(lambda row: Row(label=int(row[0]), statusid=row[1], text=row[5]))

# Create Data Frame
tweetsDF = sqlContext.createDataFrame(tweets)

# Create Bag of Words
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(tweetsDF)

# Filter Stopwords
stop = stopwords.words('english')
def filter_stopwords_and_remove_punctuation(words_array):
    out = []
    for word in words_array:
        if word not in stop:
            word_no_punct = re.sub('[^A-Za-z]+', '',word)
            if word_no_punct != "":
                out.append(word_no_punct)
    return out

# Problema: dopo questa trasformazione passa da SparkRDD a PythonRDD (WTF!?)
wordsData2 = wordsData.map(lambda row: 
                            Row(label=row.label,
                                statusid=row.statusid, 
                                text=row.text,
                                words=filter_stopwords_and_remove_punctuation(row.words)))
wordsData3= sqlContext.createDataFrame(wordsData2)

# Term Frequence
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=2000)
featurizedData = hashingTF.transform(wordsData3)

# Inverse Term Frequence
#idf = IDF(inputCol="rawFeatures", outputCol="features")
#idfModel = idf.fit(featurizedData)
#rescaledData = idfModel.transform(featurizedData)

rawFeatures=featurizedData.map(lambda x: x.rawFeatures)

**Creazione del Modello di Clustering**

In [102]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt

#Build the model (cluster the data)
clusters = KMeans.train(rawFeatures, k=2, maxIterations=10,
                        runs=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
#def error(point):
#    center = clusters.centers[clusters.predict(point)]
#    return sqrt(sum([x**2 for x in (point - center)]))

#WSSSE = rawFeatures.map(lambda point: error(point)).reduce(lambda x, y: x + y)
#print("Within Set Sum of Squared Error = " + str(WSSSE))

predicted = featurizedData.map(lambda x: Row( label=x.label,
                                              statusid=x.statusid,
                                              text=x.text,
                                              rawFeatures=x.rawFeatures,
                                              cluster=clusters.predict(x.rawFeatures)))

cluster1 = predicted.filter(lambda x: x.cluster==0).cache()
cluster2 = predicted.filter(lambda x: x.cluster==1).cache()

In [103]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

data = cluster1.map(lambda record: LabeledPoint(record.label, record.rawFeatures))
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)




Py4JJavaError: An error occurred while calling o1485.trainDecisionTreeModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 216.0 failed 1 times, most recent failure: Lost task 7.0 in stage 216.0 (TID 1076, localhost): java.lang.IllegalArgumentException: GiniAggregator given label 4.0 but requires label < numClasses (= 2).
	at org.apache.spark.mllib.tree.impurity.GiniAggregator.update(Gini.scala:94)
	at org.apache.spark.mllib.tree.impl.DTStatsAggregator.update(DTStatsAggregator.scala:100)
	at org.apache.spark.mllib.tree.DecisionTree$.orderedBinSeqOp(DecisionTree.scala:427)
	at org.apache.spark.mllib.tree.DecisionTree$.org$apache$spark$mllib$tree$DecisionTree$$nodeBinSeqOp$1(DecisionTree.scala:519)
	at org.apache.spark.mllib.tree.DecisionTree$$anonfun$org$apache$spark$mllib$tree$DecisionTree$$binSeqOp$1$1.apply(DecisionTree.scala:544)
	at org.apache.spark.mllib.tree.DecisionTree$$anonfun$org$apache$spark$mllib$tree$DecisionTree$$binSeqOp$1$1.apply(D

In [104]:
trainingData.take(10)

[LabeledPoint(0.0, (2000,[100,174,222,540,580,672,1228,1239,1278,1371,1490,1628,1894],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(0.0, (2000,[557,1222,1383,1410,1751,1945],[1.0,1.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(0.0, (2000,[13,362,616,1407,1903,1931],[1.0,1.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(0.0, (2000,[348,432,523,644,720,939,1024,1376,1655,1781],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(0.0, (2000,[545,664,761,831,927,1116,1713,1747,1858],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(0.0, (2000,[4,235,732,899,1001,1013,1029,1267,1351,1441],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(0.0, (2000,[6,141,185,478,925,1409,1701],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(0.0, (2000,[9,191,1029,1311,1466,1467,1545,1658,1685],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(0.0, (2000,[279,1013,1734],[1.0,1.0,1.0])),
 LabeledPoint(0.0, (2000,[30,187,647,1013,1178,1688,1735,1747],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, 'data/mllib/sample_libsvm_data.txt')
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification tree model:')
print(model.toDebugString())

# Save and load model
model.save(sc, "myModelPath")
sameModel = DecisionTreeModel.load(sc, "myModelPath")

In [ ]:
print("Cluster1: ")
print(cluster1.count())
print("Cluster2: ")
print(cluster2.count())
print("Record analizzati: ")
print(file.count())

In [ ]:
# Import Tweets from File (nuovi)
file=sc.textFile("/Users/fusima/GitHub/sentiment-analysis-dictionary/data/tweets-text-sample.csv")\
        .filter(lambda x: ";" in x)\
        .map(lambda line: line.split(";"))

tweets = file.map(lambda line: Row(statusid=line[0],text=line[2]))\
             .filter(lambda x: x.text != "")
